In [ ]:
from glob import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

F:\Monash\Project\archive\IDC_regular_ps50_idx5

point to the base path and show the number of patients

In [ ]:
base = "F:/Monash/Project/archive/IDC_regular_ps50_idx5/"
folder = os.listdir(base)
print(len(folder))

279


create directory if not created before

In [ ]:
img_dir = 'Breast_cancer_dir'

if os.path.isdir(img_dir):
    pass
else:
    os.mkdir(img_dir)

separate images into pos and neg, saving in different folder

In [ ]:
import shutil
pid = folder
path = "F:/Monash/Project/archive/IDC_regular_ps50_idx5/"
for i in pid:
    N = path + str(i) + '/0'
    P = path + str(i) + '/1'
    
    array0 = os.listdir(N)
    array1 = os.listdir(P)
    
    for j in array0:
        start = os.path.join(N, j)
        end = os.path.join(img_dir, j)
        shutil.copyfile(start, end)
        
    for k in array1:
        start = os.path.join(P, k)
        end = os.path.join(img_dir, k)
        shutil.copyfile(start, end)

In [ ]:
img_dir = os.listdir('Breast_cancer_dir')
data = pd.DataFrame(img_dir, columns = ['img_id'])

def lab(ind):
    l = ind.split('_')
    d = l[4] 
    return d[5]

def pid(ind):
    id = ind.split('_')
    return id[0]

In [ ]:
data['pid'] = data['img_id'].apply(pid)
data['lab'] = data['img_id'].apply(lab)

In [ ]:
data.head()

,img_id,pid,lab
0,10253_idx5_x1001_y1001_class0.png,10253,0
1,10253_idx5_x1001_y1051_class0.png,10253,0
2,10253_idx5_x1001_y1101_class0.png,10253,0
3,10253_idx5_x1001_y1151_class0.png,10253,0
4,10253_idx5_x1001_y1201_class0.png,10253,0


In [ ]:
data.tail()

,img_id,pid,lab
277519,9383_idx5_x951_y401_class0.png,9383,0
277520,9383_idx5_x951_y651_class0.png,9383,0
277521,9383_idx5_x951_y801_class0.png,9383,0
277522,9383_idx5_x951_y901_class0.png,9383,0
277523,9383_idx5_x951_y951_class0.png,9383,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data, test_size = 0.2, stratify = data['lab'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, test_size=0.2, random_state=1, stratify = data['lab'])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [ ]:
X_train.shape

(222019, 3)

In [ ]:
X_test.shape

(55505, 3)

create train and valid directory to batch process images

previous attempt had memory error as pc cannot allocate 68GB in a go

resolve attempt >>> call from directory
https://stackoverflow.com/questions/62991747/memory-error-while-training-my-model-unable-to-allocate-31-9-gib-for-an-array-w
https://www.geeksforgeeks.org/cnn-image-data-pre-processing-with-generators/

In [ ]:
ori = 'origin'
os.mkdir(ori)

# Train directory
train = os.path.join(ori, 'train')
os.mkdir(train)
# pos and neg train data
pos_train = os.path.join(train, 'pos_train')
neg_train = os.path.join(train, 'neg_train')
os.mkdir(pos_train)
os.mkdir(neg_train)

# Valid directory
test = os.path.join(ori, 'test')
os.mkdir(test)
# pos and neg valid data
pos_test = os.path.join(test, 'pos_test')
neg_test = os.path.join(test, 'neg_test')
os.mkdir(pos_test)
os.mkdir(neg_test)


In [ ]:
os.listdir('origin/train')

['neg_train', 'pos_train']

In [ ]:
data['img_id']

0         10253_idx5_x1001_y1001_class0.png
1         10253_idx5_x1001_y1051_class0.png
2         10253_idx5_x1001_y1101_class0.png
3         10253_idx5_x1001_y1151_class0.png
4         10253_idx5_x1001_y1201_class0.png
5         10253_idx5_x1001_y1251_class0.png
6         10253_idx5_x1001_y1301_class0.png
7         10253_idx5_x1001_y1351_class0.png
8         10253_idx5_x1001_y1501_class0.png
9         10253_idx5_x1001_y1551_class0.png
10        10253_idx5_x1001_y1701_class0.png
11         10253_idx5_x1001_y351_class0.png
12         10253_idx5_x1001_y401_class0.png
13         10253_idx5_x1001_y451_class0.png
14         10253_idx5_x1001_y501_class0.png
15         10253_idx5_x1001_y551_class0.png
16         10253_idx5_x1001_y601_class0.png
17         10253_idx5_x1001_y651_class0.png
18         10253_idx5_x1001_y701_class0.png
19         10253_idx5_x1001_y751_class0.png
20         10253_idx5_x1001_y801_class0.png
21         10253_idx5_x1001_y951_class0.png
22        10253_idx5_x1051_y1001

In [ ]:
data.set_index('img_id', inplace =True)

In [ ]:
def transfer(d, directory, Data, typ):
    direct = str(typ) 
    for i in d:
        try:
            ind = i
            label = Data.loc[i, 'lab']
            if label == '0':
                end = os.path.join(directory, 'neg_'+direct, ind)
            else:
                end = os.path.join(directory, 'pos_'+direct, ind)
            start = os.path.join('breast_cancer_dir', ind)
            shutil.move(start, end)
        except:
            continue

In [ ]:
Train = list(X_train['img_id'])
Test = list(X_test['img_id'])

transfer(Train, train, data, 'train')
transfer(Test, test, data, 'test')

In [ ]:
os.listdir("origin/train/pos_train")

['10253_idx5_x501_y351_class1.png',
 '10253_idx5_x501_y401_class1.png',
 '10253_idx5_x551_y301_class1.png',
 '10253_idx5_x551_y351_class1.png',
 '10253_idx5_x551_y401_class1.png',
 '10253_idx5_x551_y451_class1.png',
 '10253_idx5_x551_y651_class1.png',
 '10253_idx5_x551_y701_class1.png',
 '10253_idx5_x601_y251_class1.png',
 '10253_idx5_x601_y351_class1.png',
 '10253_idx5_x601_y601_class1.png',
 '10253_idx5_x601_y651_class1.png',
 '10253_idx5_x601_y701_class1.png',
 '10253_idx5_x601_y751_class1.png',
 '10253_idx5_x651_y251_class1.png',
 '10253_idx5_x651_y301_class1.png',
 '10253_idx5_x651_y351_class1.png',
 '10253_idx5_x651_y401_class1.png',
 '10253_idx5_x651_y451_class1.png',
 '10253_idx5_x651_y601_class1.png',
 '10253_idx5_x651_y701_class1.png',
 '10253_idx5_x651_y751_class1.png',
 '10253_idx5_x701_y251_class1.png',
 '10253_idx5_x701_y301_class1.png',
 '10253_idx5_x701_y351_class1.png',
 '10253_idx5_x701_y401_class1.png',
 '10253_idx5_x701_y601_class1.png',
 '10253_idx5_x701_y651_class

In [ ]:
os.listdir('origin/test/neg_test')

['10253_idx5_x1001_y1251_class0.png',
 '10253_idx5_x1001_y401_class0.png',
 '10253_idx5_x1051_y1051_class0.png',
 '10253_idx5_x1051_y351_class0.png',
 '10253_idx5_x1101_y1751_class0.png',
 '10253_idx5_x1101_y1801_class0.png',
 '10253_idx5_x1101_y351_class0.png',
 '10253_idx5_x1101_y451_class0.png',
 '10253_idx5_x1101_y551_class0.png',
 '10253_idx5_x1101_y601_class0.png',
 '10253_idx5_x1101_y901_class0.png',
 '10253_idx5_x1101_y951_class0.png',
 '10253_idx5_x1151_y1151_class0.png',
 '10253_idx5_x1151_y1901_class0.png',
 '10253_idx5_x1151_y201_class0.png',
 '10253_idx5_x1151_y401_class0.png',
 '10253_idx5_x1151_y451_class0.png',
 '10253_idx5_x1151_y501_class0.png',
 '10253_idx5_x1151_y701_class0.png',
 '10253_idx5_x1201_y1001_class0.png',
 '10253_idx5_x1201_y1151_class0.png',
 '10253_idx5_x1201_y751_class0.png',
 '10253_idx5_x1251_y1501_class0.png',
 '10253_idx5_x1251_y751_class0.png',
 '10253_idx5_x1301_y1051_class0.png',
 '10253_idx5_x1301_y701_class0.png',
 '10253_idx5_x1301_y901_clas

In [ ]:
'origin/train/pos_train'

'origin/train/pos_train'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale = 1./255)

datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.25,
                            rotation_range=10, width_shift_range=0.2,
                            height_shift_range=0.2, zoom_range=0.2, 
                            horizontal_flip=True)


train_datagen = datagen.flow_from_directory('origin/train', target_size=(50, 50),
                                           batch_size=20,
                                            subset = 'training',
                                           class_mode = 'categorical')

val_datagen = datagen.flow_from_directory('origin/train', target_size=(50, 50),
                                           batch_size=20,
                                           class_mode = 'categorical',
                                               subset = 'validation')

test_datagen = test_datagen.flow_from_directory('origin/test', target_size=(50, 50),
                                           batch_size=1,
                                           class_mode = 'categorical',
                                          shuffle = False) 

C:\Users\Acer\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Acer\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Acer\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Acer\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:5

Found 166515 images belonging to 2 classes.
Found 55504 images belonging to 2 classes.
Found 55505 images belonging to 2 classes.


Notes:
After looking up stackoverflow, there is a way to split the dataset to train and valid
so the original valid folder will be changed to test data

https://stackoverflow.com/questions/53037510/can-flow-from-directory-get-train-and-validation-data-from-the-same-directory-in

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

![image.png](attachment:image.png)

In [ ]:
model = Sequential()
model.add(Convolution2D(64, (3,3), activation = 'relu', input_shape = (50,50,3)))
model.add(Convolution2D(64, (3, 3), activation = 'relu'))
model.add(Convolution2D(64, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Convolution2D(64, (3, 3), activation = 'relu'))
model.add(Convolution2D(64, (3, 3), activation = 'relu'))
model.add(Convolution2D(64, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Convolution2D(64, (3, 3), activation = 'relu'))
model.add(Convolution2D(64, (3, 3), activation = 'relu'))
model.add(Convolution2D(64, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

In [ ]:
model.build()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 48, 48, 64)        1792      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_6 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 20, 20, 64)       

In [ ]:
model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_datagen, validation_data= val_datagen, steps_per_epoch = 177616//64, validation_steps= 44403//64, epochs=20)

Epoch 1/20
2775/2775 [==============================] - 1357s 488ms/step - loss: 0.4151 - accuracy: 0.8234 - val_loss: 0.5724 - val_accuracy: 0.7629
Epoch 2/20
2775/2775 [==============================] - 1304s 470ms/step - loss: 0.3946 - accuracy: 0.8315 - val_loss: 3.2717 - val_accuracy: 0.7217
Epoch 3/20
2775/2775 [==============================] - 1376s 496ms/step - loss: 0.3766 - accuracy: 0.8388 - val_loss: 0.6255 - val_accuracy: 0.6802
Epoch 4/20
2775/2775 [==============================] - 1341s 483ms/step - loss: 0.3521 - accuracy: 0.8519 - val_loss: 0.7851 - val_accuracy: 0.2831
Epoch 5/20
2775/2775 [==============================] - 1344s 484ms/step - loss: 0.3492 - accuracy: 0.8533 - val_loss: 0.4889 - val_accuracy: 0.7809
Epoch 6/20
2775/2775 [==============================] - 1326s 478ms/step - loss: 0.3368 - accuracy: 0.8590 - val_loss: 0.6289 - val_accuracy: 0.6496
Epoch 7/20
2775/2775 [==============================] - 1294s 466ms/step - loss: 0.3314 - accuracy: 0.8609

In [ ]:
model.save('C:/Users/Acer/Desktop/Semester 2/FIT3164/CNN', save_format = 'tf')

INFO:tensorflow:Assets written to: C:/Users/Acer/Desktop/Semester 2/FIT3164/CNN\assets


In [ ]:
predict = model.evaluate_generator(test_datagen, verbose = 1)

    6/55505 [..............................] - ETA: 9:57 - loss: 0.0233 - accuracy: 1.0000       

C:\Users\Acer\Anaconda3\lib\site-packages\keras\engine\training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


55505/55505 [==============================] - 452s 8ms/step - loss: 0.3456 - accuracy: 0.8586


model.save('C:/Users/Acer/Desktop/Semester 2/FIT3164/Img_recognition', save_format = 'tf')

In [ ]:
predict[1]

0.8585892915725708

In [ ]:
import numpy as np
import cv2 as cv

# img = Image.open('C:/Users/Acer/origin/train/pos_train/8863_idx5_x1001_y801_class1.png')
img = cv.imread('C:/Users/Acer/origin/train/pos_train/10253_idx5_x601_y651_class1.png')
# img = cv.imread('C:/Users/Acer/origin/train/neg_train/8863_idx5_x101_y1201_class0.png')
image = cv.resize(img, (50, 50))
image_tensor = tf.cast(img, tf.float32)
image_tensor = image_tensor / 255.
imgs = image_tensor[None, :]
pred = model.predict(imgs)
p = np.argmax(pred, axis=1)
if p[0] == 0:
  print("No Cancer")
elif p[1] == 1:
  print("cancer")

[[0.42558026 0.5744198 ]]
